<div style="background:#5D6D7E;padding:20px;color:#ffffff;margin-top:10px;">

# NLP - Práctica 3 (Clasificación de Textos) 

## Profesora: Lisibonny Beato
### Período 3-2023-2024</div>


In [1]:
# Importando algunas librerias

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.feature_extraction import _stop_words
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay, classification_report

<div style="background:#ff6242;padding:20px;color:#ffffff;margin-top:10px;">
<b>El propósito de esta asignación es que el estudiante cree varios modelos de clasificación sobre un corpus textual, evaluando el desempeño de cada uno y seleccionando el que mejor de respuesta a la tarea<br />
<br />
    Para esta práctica estarán utilizando el dataset <i>reviews_rotten_tomatoes.csv</i> que encontrarán en la carpeta correspondiente de la PVA.
<br />
<br />
Es importante que realicen un análisis detallado para cada modelo construido y que realicen una comparativa final, argumentando adecuadamente las razones de que se elija finalmente uno de los modelos sobre los otros.</b>
</div>


## 1. Nivel Básico 
### Puntuación máxima de la tarea: 5 puntos
#### 1.1 Limpieza y preparación de los datos, tomando en cuenta que el dataset continue una valoración sentimental que los usuarios hacen acerca de películas en el sitio RottenTomatoes. 
#### 1.2 Representar los documentos como vectores BOW
#### 1.3 Construcción de un modelo de clasificación bayesiano
#### 1.4 Construcción de un modelo basado en regresión logística
#### 1.5 Construcción de un modelo basado en árboles decisorios 
#### Nota: Para los todos los modelos debe realizar un análisis de las métricas de desempeño, sobre los modelos construidos con una partición de los datos para training y otra para testing.

<div style="background:#FFFFE0;padding:20px;color:#000000;margin-top:10px;">
Utilice esta celda para colocar comentarios en el notebook, cuando lo estime necesario. Copiela varias veces donde considere.
</div>

## 2. Nivel Intermedio 
### Puntuación máxima de la tarea: 3 puntos
#### 2.1 Investigue acerca del concepto de  validación cruzada en clasificación y escriba sobre el tema en no más de dos párrafos (adicionalmente, puede colocar una imagen, si así lo desea)
#### 2.2 Investigue las funciones de la librería Scikit Learn que ayudan a realizar validación cruzada y descríbalas en detalle.
#### 2.3 Construya los mismos modelos del punto anterior, pero ahora con un esquema de prueba basado en validación cruzada y realice un análisis de las métricas de desempeño de cada uno

## 3. Nivel Avanzado
### Puntuación máxima de la tarea: 2 puntos
#### Investigue sobre el roc auc score y grafíquelo para el mejor modelo de los 6 que ha construido. Interprete los resultados que se visualizan.